In [185]:
import pandas as pd
import sqlite3

In [186]:
orders = pd.read_csv('/home/vitsli/DataAnalitycs/tests/easybot/orders.csv')
order_items = pd.read_csv('/home/vitsli/DataAnalitycs/tests/easybot/order_items.csv')
products = pd.read_csv('/home/vitsli/DataAnalitycs/tests/easybot/products.csv')
customers = pd.read_csv('/home/vitsli/DataAnalitycs/tests/easybot/customers.csv')
categories = pd.read_csv('/home/vitsli/DataAnalitycs/tests/easybot/categories.csv')

In [187]:
orders.head()

,order_id,customer_id,order_date,payment_method,total_amount
0,1,116,2023-04-02,PayPal,850
1,3,113,2023-04-05,PayPal,570
2,4,116,2023-04-18,Credit Card,400
3,5,123,2023-04-07,Apple Pay,260
4,6,111,2023-03-29,Apple Pay,180


In [188]:
order_items.head()

,item_id,order_id,product_id,quantity,price
0,1,1,1044,3,70
1,2,1,1030,3,50
2,3,1,1033,5,50
3,4,1,1039,3,80
4,5,3,1018,5,60


In [189]:
order_items.shape

(250, 5)

In [190]:
products.head()

,product_id,category_id,product_name,stock
0,1001,2,Product 0,190
1,1002,1,Product 1,140
2,1003,2,Product 2,220
3,1004,3,Product 3,180
4,1005,2,Product 4,50


In [191]:
products.category_id.unique()

array([2, 1, 3])

In [192]:
customers.head()

,customer_id,customer_name,registration_date,email
0,101,Lane Ward,2023-03-15,lane.ward@example.com
1,102,Cole Chandler,2023-04-13,cole.chandler@example.com
2,103,Toby Jimenez,2023-03-12,toby.jimenez@example.com
3,104,Christine Stephenson,2023-03-14,christine.stephenson@example.com
4,105,Zelda Horton,2023-03-04,zelda.horton@example.com


In [193]:
categories.head()

,category_id,category_name
0,1,Apparel
1,2,Electronics
2,3,Goods


In [194]:
con = sqlite3.connect('db')

In [195]:
cur = con.cursor()

In [196]:
orders.to_sql('orders', con, index=False, if_exists='replace')

122

In [197]:
customers.to_sql('customers', con, index=False, if_exists='replace')

30

In [198]:
products.to_sql('products', con, index=False, if_exists='replace')

49

In [199]:
order_items.to_sql('order_items', con, index=False, if_exists='replace')

250

In [200]:
categories.to_sql('categories', con, index=False, if_exists='replace')

3

### 1.1. Количество уникальных пользователей за последние 30 дней.

In [251]:
sql = '''
SELECT COUNT(DISTINCT customer_id) AS customers_cnt
FROM orders
WHERE order_date >= (SELECT date(MAX(order_date), '-30 Days') FROM orders)'''

In [253]:
select(sql)

,customers_cnt
0,30


In [252]:
# # На Postgre
# SELECT COUNT(DISTINCT customer_id) AS customers_cnt
# FROM orders
# WHERE order_date >= (SELECT MAX(order_date) - interval '30 Days' FROM orders)

### 1.2. Количество покупок и общая сумма продаж за последние 30 дней.

In [203]:
def select(sql):
  return pd.read_sql(sql,con)

In [205]:
sql = '''
SELECT
    COUNT(order_id) as order_cnt,
    SUM(total_amount) as revenue
FROM orders
WHERE order_date >= (SELECT date(MAX(order_date), '-30 Days') FROM orders)
'''

In [206]:
select(sql)

,order_cnt,revenue
0,122,51030


In [209]:
# На Postgre
# SELECT
#     COUNT(order_id) as order_cnt,
#     SUM(total_amount) as revenue
# FROM orders
# WHERE order_date >= (SELECT MAX(order_date) - interval '30 Days' FROM orders)

### 1.3 Средний чек и средний LTV для каждой категории продуктов

Для рассчета LTV наиболее подходящей в данном случае кажется формула LTV = AOV * Lifetime * RPR, где AOV - средний чек, Lifetime - время жизни клиента в днях (считается как интеграл от Retention rate), RPR - коэффициент повторных покупок. Поскольку данные синтетические датой 'рождения' клиента считаю дату первой покупки, а не регистрации(иначе выйдет несуразица). 

In [230]:
sql = '''
WITH assembled_tab as (
        SELECT 
            t1.order_id,
            t1.product_id,
            t2.customer_id,
            t2.order_date,
            t1.quantity,
            t1.price,
            t4.category_name,
            MIN(t2.order_date) OVER (PARTITION BY customer_id) AS start_date
        FROM order_items t1
        LEFT JOIN orders t2
            ON t1.order_id = t2.order_id
        LEFT JOIN products t3
            ON t1.product_id = t3.product_id
        LEFT JOIN categories t4
            ON t3.category_id = t4.category_id),

        lifetime AS (            
        SELECT SUM (t3.retention) AS lt
        FROM ( 
            SELECT 
                t2.diff,
                (t2.cnt_users * 1.0)/FIRST_VALUE(t2.cnt_users) OVER (ORDER BY t2.diff) AS retention
            FROM (    
                SELECT 
                    t1.diff,
                    COUNT(DISTINCT t1.customer_id) AS cnt_users
                FROM (    
                    SELECT 
                        customer_id,
                        julianday(order_date) - julianday(start_date) as diff
                    FROM assembled_tab
                    GROUP BY customer_id) t1
                GROUP BY t1.diff) t2
                ) t3),
                
        repeat_rate as (
        SELECT     
            SUM(CASE WHEN t1.orders_quantity > 1 THEN 1 ELSE 0 END) * 1.0 / COUNT (t1.customer_id) AS rpr
        FROM (        
            SELECT
                customer_id,
                COUNT(DISTINCT order_id) AS orders_quantity
            FROM orders
            GROUP BY customer_id) t1
            )


SELECT
    category_name,
    SUM(quantity * price) / COUNT(DISTINCT order_id) AS aov,
    ROUND(SUM(quantity * price) / COUNT(DISTINCT order_id) * lt * rpr, 2) AS ltv
FROM assembled_tab, repeat_rate, lifetime
GROUP BY category_name
    
'''  

In [231]:
select(sql)

,category_name,aov,ltv
0,Apparel,320,1031.11
1,Electronics,202,650.89
2,Goods,271,873.22


In [212]:
# Postgre
# WITH assembled_tab as (
#         SELECT 
#             t1.order_id,
#             t1.product_id,
#             t2.customer_id,
#             t2.order_date,
#             t1.quantity,
#             t1.price,
#             t4.category_name,
#             MIN(t2.order_date) OVER (PARTITION BY customer_id) AS start_date
#         FROM order_items t1
#         LEFT JOIN orders t2
#             ON t1.order_id = t2.order_id
#         LEFT JOIN products t3
#             ON t1.product_id = t3.product_id
#         LEFT JOIN categories t4
#             ON t3.category_id = t4.category_id),

#         lifetime AS (            
#         SELECT SUM (t3.retention) AS lt
#         FROM ( 
#             SELECT 
#                 t2.diff,
#                 (t2.cnt_users * 1.0)/FIRST_VALUE(t2.cnt_users) OVER (ORDER BY t2.diff) AS retention
#             FROM (    
#                 SELECT 
#                     t1.diff,
#                     COUNT(DISTINCT t1.customer_id) AS cnt_users
#                 FROM (    
#                     SELECT 
#                         customer_id,
#                         order_date - start_date as diff
#                     FROM assembled_tab
#                     GROUP BY customer_id) t1
#                 GROUP BY t1.diff) t2
#                 ) t3),
                
#         repeat_rate as (
#         SELECT     
#             SUM(CASE WHEN t1.orders_quantity > 1 THEN 1 ELSE 0 END) * 1.0 / COUNT (t1.customer_id) AS rpr
#         FROM (        
#             SELECT
#                 customer_id,
#                 COUNT(DISTINCT order_id) AS orders_quantity
#             FROM orders
#             GROUP BY customer_id) t1
#             )


# SELECT
#     category_name,
#     SUM(quantity * price) / COUNT(DISTINCT order_id) AS aov,
#     SUM(quantity * price) / COUNT(DISTINCT order_id) * lt * rpr AS ltv
# FROM assembled_tab, repeat_rate, lifetime
# GROUP BY category_name

In [213]:
sql = '''
 SELECT 
            t1.order_id,
            t1.product_id,
            t2.customer_id,
            t2.order_date,
            t1.quantity,
            t1.price,
            t4.category_name,
            MIN(t2.order_date) OVER (PARTITION BY customer_id) AS start_date
        FROM order_items t1
        LEFT JOIN orders t2
            ON t1.order_id = t2.order_id
        LEFT JOIN products t3
            ON t1.product_id = t3.product_id
        LEFT JOIN categories t4
            ON t3.category_id = t4.category_id
'''

In [214]:
select(sql)

,order_id,product_id,customer_id,order_date,quantity,price,category_name,start_date
0,32,1039,101,2023-03-25,2,80,Apparel,2023-03-25
1,32,1043,101,2023-03-25,2,80,Goods,2023-03-25
2,32,1018,101,2023-03-25,1,70,Apparel,2023-03-25
3,104,1005,101,2023-04-07,1,90,Electronics,2023-03-25
4,104,1012,101,2023-04-07,4,70,Apparel,2023-03-25
...,...,...,...,...,...,...,...,...
245,126,1040,130,2023-04-03,2,50,Goods,2023-03-25
246,144,1017,130,2023-03-29,4,80,Apparel,2023-03-25
247,144,1004,130,2023-03-29,4,70,Goods,2023-03-25
248,146,1039,130,2023-03-29,4,90,Apparel,2023-03-25


### 1.4. Топ-5 продуктов по количеству покупок и выручке за последние 30 дней.

In [215]:
sql = '''
SELECT 
    t1.product_id,
    COUNT(DISTINCT t1.order_id) AS purchases_amount,
    SUM(t1.quantity * t1.price) AS revenue
    
FROM (    
    SELECT 
        t1.order_id,
        t1.product_id,
        t1.quantity,
        t1.price,
        t2.order_date    
    FROM order_items t1
    INNER JOIN orders t2
        ON t1.order_id = t2.order_id 
        AND t2.order_date >= (SELECT date(MAX(order_date), '-30 Days') FROM orders)
        ) t1
GROUP BY t1.product_id 
ORDER BY purchases_amount DESC, revenue DESC
LIMIT 5
'''

In [216]:
select(sql)

,product_id,purchases_amount,revenue
0,1039,11,2600
1,1005,10,2240
2,1040,9,1800
3,1009,9,1360
4,1007,8,2200


In [217]:
# На Postgre

# SELECT 
#     t1.product_id,
#     COUNT(DISTINCT t1.order_id) AS purchases_amount,
#     SUM(t1.quantity * t1.price) AS revenue
    
# FROM (    
#     SELECT 
#         t1.order_id,
#         t1.product_id,
#         t1.quantity,
#         t1.price,
#         t2.order_date    
#     FROM order_items t1
#     INNER JOIN orders t2
#         ON t1.order_id = t2.order_id 
#         AND t2.order_date >= (SELECT MAX(order_date) - interval '30 Days' FROM orders)
#         ) t1
# GROUP BY t1.product_id 
# ORDER BY purchases_amount DESC, revenue DESC
# LIMIT 5


### 1.5. Сегментировать пользователей на группы по частоте использования продукта и выявить основные различия между этими группами.

In [ ]:
# Запросы для Postgre будут такими же

In [218]:
sql = '''
SELECT 
    t1.item_id,
    t1.order_id,
    t1.product_id,
    t3.product_name,
    t1.price,
    t1.quantity,
    t2.customer_id,
    t2.user_group,
    t2.order_date,
    t2.payment_method,
    t4.category_name    
FROM order_items t1
LEFT JOIN (
    SELECT 
        *,
        CASE 
            WHEN COUNT(order_id) OVER (PARTITION BY customer_id) BETWEEN 0 AND 3 THEN 'low'
            WHEN COUNT(order_id) OVER (PARTITION BY customer_id) BETWEEN 3 AND 6 THEN 'middle'
            ELSE 'hi'
        END user_group    
    FROM orders) t2
    ON t1.order_id = t2.order_id
LEFT JOIN products t3
    ON t1.product_id = t3.product_id
LEFT JOIN categories t4
    ON t3.category_id = t4.category_id

'''

In [219]:
select(sql).head(10)

,item_id,order_id,product_id,product_name,price,quantity,customer_id,user_group,order_date,payment_method,category_name
0,1,1,1044,Product 43,70,3,116,low,2023-04-02,PayPal,Goods
1,2,1,1030,Product 29,50,3,116,low,2023-04-02,PayPal,Apparel
2,3,1,1033,Product 32,50,5,116,low,2023-04-02,PayPal,Goods
3,4,1,1039,Product 38,80,3,116,low,2023-04-02,PayPal,Apparel
4,5,3,1018,Product 17,60,5,113,low,2023-04-05,PayPal,Apparel
5,6,3,1019,Product 18,90,3,113,low,2023-04-05,PayPal,Electronics
6,7,4,1047,Product 46,80,5,116,low,2023-04-18,Credit Card,Goods
7,8,5,1021,Product 20,60,1,123,middle,2023-04-07,Apple Pay,Electronics
8,9,5,1016,Product 15,50,4,123,middle,2023-04-07,Apple Pay,Apparel
9,10,6,1008,Product 7,90,2,111,hi,2023-03-29,Apple Pay,Apparel


In [236]:
sql = '''
WITH assembled_tab AS (
    SELECT 
        t1.item_id,
        t1.order_id,
        t1.product_id,
        t3.product_name,
        t1.price,
        t1.quantity,
        t2.customer_id,
        t2.user_group,
        t2.order_date,
        t2.payment_method,
        t4.category_name    
    FROM order_items t1
    LEFT JOIN (
        SELECT 
            *,
            CASE 
                WHEN COUNT(order_id) OVER (PARTITION BY customer_id) BETWEEN 1 AND 3 THEN 'low'
                WHEN COUNT(order_id) OVER (PARTITION BY customer_id) BETWEEN 4 AND 6 THEN 'middle'
                ELSE 'hi'                
            END user_group    
        FROM orders) t2
        ON t1.order_id = t2.order_id
    LEFT JOIN products t3
        ON t1.product_id = t3.product_id
    LEFT JOIN categories t4
        ON t3.category_id = t4.category_id)

SELECT 
    user_group,
    COUNT(DISTINCT customer_id) AS customers_quantity,
    SUM(price * quantity) / COUNT(DISTINCT customer_id) AS arppu
FROM assembled_tab
GROUP BY user_group
'''

In [237]:
select(sql)

,user_group,customers_quantity,arppu
0,hi,4,3480
1,low,15,1174
2,middle,11,1772


In [242]:
sql = '''
WITH assembled_tab AS (
    SELECT 
        t1.item_id,
        t1.order_id,
        t1.product_id,
        t3.product_name,
        t1.price,
        t1.quantity,
        t2.customer_id,
        t2.user_group,
        t2.order_date,
        t2.payment_method,
        t4.category_name    
    FROM order_items t1
    LEFT JOIN (
        SELECT 
            *,
            CASE 
                WHEN COUNT(order_id) OVER (PARTITION BY customer_id) BETWEEN 0 AND 3 THEN 'low'
                WHEN COUNT(order_id) OVER (PARTITION BY customer_id) BETWEEN 3 AND 6 THEN 'middle'
                ELSE 'hi'
            END user_group    
        FROM orders) t2
        ON t1.order_id = t2.order_id
    LEFT JOIN products t3
        ON t1.product_id = t3.product_id
    LEFT JOIN categories t4
        ON t3.category_id = t4.category_id)
SELECT 
    t1.user_group,
    t1.category_name,
    ROUND(t1.purchases_amount * 100.0 / SUM(t1.purchases_amount) 
        OVER (PARTITION BY t1.user_group), 2) AS purchase_percentage
FROM (

    SELECT 
        user_group,
        category_name,
        COUNT(DISTINCT order_id) AS purchases_amount   
    FROM assembled_tab
    GROUP BY user_group, category_name) t1
    ORDER BY user_group, purchase_percentage DESC
'''

In [243]:
select(sql)

,user_group,category_name,purchase_percentage
0,hi,Goods,40.00
1,hi,Apparel,34.00
2,hi,Electronics,26.00
3,low,Apparel,43.33
4,low,Goods,33.33
5,low,Electronics,23.33
6,middle,Apparel,36.71
7,middle,Electronics,35.44
8,middle,Goods,27.85


In [244]:
sql = '''
WITH assembled_tab AS (
    SELECT 
        t1.item_id,
        t1.order_id,
        t1.product_id,
        t3.product_name,
        t1.price,
        t1.quantity,
        t2.customer_id,
        t2.user_group,
        t2.order_date,
        t2.payment_method,
        t4.category_name    
    FROM order_items t1
    LEFT JOIN (
        SELECT 
            *,
            CASE 
                WHEN COUNT(order_id) OVER (PARTITION BY customer_id) BETWEEN 0 AND 3 THEN 'low'
                WHEN COUNT(order_id) OVER (PARTITION BY customer_id) BETWEEN 3 AND 6 THEN 'middle'
                ELSE 'hi'
            END user_group    
        FROM orders) t2
        ON t1.order_id = t2.order_id
    LEFT JOIN products t3
        ON t1.product_id = t3.product_id
    LEFT JOIN categories t4
        ON t3.category_id = t4.category_id)
SELECT 
    t1.user_group,
    t1.payment_method,
    ROUND(t1.purchases_amount * 100.0 / SUM(t1.purchases_amount) 
        OVER (PARTITION BY t1.user_group), 2) AS purchase_percentage
FROM (
    SELECT 
        user_group,
        payment_method,
        COUNT(DISTINCT order_id) AS purchases_amount   
    FROM assembled_tab
    GROUP BY user_group, payment_method) t1
ORDER BY user_group ASC, purchase_percentage DESC
'''

In [245]:
select(sql)

,user_group,payment_method,purchase_percentage
0,hi,PayPal,38.71
1,hi,Apple Pay,32.26
2,hi,Google Pay,16.13
3,hi,Credit Card,12.90
4,low,PayPal,37.84
5,low,Apple Pay,29.73
6,low,Credit Card,21.62
7,low,Google Pay,10.81
8,middle,Google Pay,29.63
9,middle,Credit Card,27.78
